In [1]:
# Dependencies and Setup
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session

from config import username, password, host, database

## Read in CSV files

In [39]:
# File path
race_filepath = '../../../machine_learning/Resources/race_data.csv'
non_race_filepath = '../../../machine_learning/Resources/non_race_data.csv'

# Read in csv file
race_df = pd.read_csv(race_filepath)
non_race_df = pd.read_csv(non_race_filepath)

In [5]:
race_df.head()

,state,state_code,year,race,hispanic_origin,births_by_race,deaths_by_race,mmr_by_race,population_by_race,id,state_abbv,latitude,longitude
0,Alabama,1,2016,Black or African American,Not Hispanic or Latino,17989,14,77.83,0,US.AL,AL,32.318231,-86.902298
1,Alabama,1,2016,White,Not Hispanic or Latino,35319,18,50.96,0,US.AL,AL,32.318231,-86.902298
2,Alabama,1,2017,Black or African American,Not Hispanic or Latino,18354,25,136.21,704201,US.AL,AL,32.318231,-86.902298
3,Alabama,1,2017,White,Not Hispanic or Latino,34784,15,43.12,1655605,US.AL,AL,32.318231,-86.902298
4,Alabama,1,2018,Black or African American,Not Hispanic or Latino,17939,14,78.04,706754,US.AL,AL,32.318231,-86.902298


In [6]:
race_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   state               388 non-null    object 
 1   state_code          388 non-null    int64  
 2   year                388 non-null    int64  
 3   race                388 non-null    object 
 4   hispanic_origin     388 non-null    object 
 5   births_by_race      388 non-null    int64  
 6   deaths_by_race      388 non-null    int64  
 7   mmr_by_race         388 non-null    float64
 8   population_by_race  388 non-null    int64  
 9   id                  388 non-null    object 
 10  state_abbv          388 non-null    object 
 11  latitude            388 non-null    float64
 12  longitude           388 non-null    float64
dtypes: float64(3), int64(5), object(5)
memory usage: 39.5+ KB


In [7]:
non_race_df.head()

,year,state,id,state_code,latitude,longitude,deaths,births,maternal_mortality_ratio,population,...,prem_death_val,smoking_val,uninsured_val,all_determs_val,all_outcomes_val,chlamydia_val,prem_death_ri_val,teen_birth_val,primary_care_val,low_birthweight_val
0,2015,Alabama,US.AL,AL,32.318231,-86.902298,12.0,59657.0,20.11,2505795.0,...,10095.0,21.1,12.9,-0.371,-0.325,611.0,1.2,34.3,103.5,10.0
1,2016,Alabama,US.AL,AL,32.318231,-86.902298,35.0,59151.0,59.17,2507714.0,...,10097.0,21.4,11.1,-0.427,-0.366,600.2,1.1,32.0,116.4,10.1
2,2017,Alabama,US.AL,AL,32.318231,-86.902298,41.0,58941.0,69.56,2514911.0,...,10321.0,21.5,9.6,-0.427,-0.335,543.6,1.2,30.1,119.3,10.4
3,2018,Alabama,US.AL,AL,32.318231,-86.902298,31.0,57761.0,53.67,2523756.0,...,10720.0,20.9,9.3,-0.483,-0.356,553.6,1.1,28.4,122.8,10.3
4,2019,Alabama,US.AL,AL,32.318231,-86.902298,35.0,58615.0,59.71,2533668.0,...,10435.0,19.2,9.7,-0.437,-0.383,614.1,1.2,27.0,122.8,10.3


In [41]:
non_race_df.describe()['maternal_mortality_ratio']

count    308.000000
mean      31.339578
std       13.112060
min        8.170000
25%       22.142500
50%       29.045000
75%       38.087500
max       88.640000
Name: maternal_mortality_ratio, dtype: float64

In [8]:
non_race_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308 entries, 0 to 307
Data columns (total 42 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   year                      308 non-null    int64  
 1   state                     308 non-null    object 
 2   id                        308 non-null    object 
 3   state_code                308 non-null    object 
 4   latitude                  308 non-null    float64
 5   longitude                 308 non-null    float64
 6   deaths                    308 non-null    float64
 7   births                    308 non-null    float64
 8   maternal_mortality_ratio  308 non-null    float64
 9   population                308 non-null    float64
 10  employer                  308 non-null    float64
 11  non_group                 308 non-null    float64
 12  medicaid                  308 non-null    float64
 13  medicare                  308 non-null    float64
 14  military  

In [2]:
avg_data_df = pd.read_csv("../../../machine_learning/Resources/combined_avg_2009_2019.csv", 
                          usecols=['year', 'maternal_mortality_ratio'])[['year', 'maternal_mortality_ratio']]
avg_data_df.head()

,year,maternal_mortality_ratio
0,2009,26.75
1,2010,25.72
2,2011,27.87
3,2012,29.93
4,2013,31.73


In [3]:
avg_data = avg_data_df.append({'year': "2020", 
                            'maternal_mortality_ratio': float(0.00)}, 
                           ignore_index=True)
avg_data

,year,maternal_mortality_ratio
0,2009,26.75
1,2010,25.72
2,2011,27.87
3,2012,29.93
4,2013,31.73
5,2014,32.34
6,2015,31.94
7,2016,35.44
8,2017,34.21
9,2018,31.53


In [23]:
avg_data.to_csv('../../../machine_learning/Resources/user_input.csv', index=False)

In [31]:
forecast_data = pd.read_csv('../../../machine_learning/Resources/prediction_api_df.csv')
forecast_data.head()

,year,mmr_prediction,maternal_mortality_ratio,diabetes_val,prem_death_val,phys_inac_val,low_birthweight_val,obesity_val,cardio_death_val,medicare,...,military,uninsured,air_pollution_val,choles_check_val,drug_deaths_val,immun_child_val,infect_dis_val,uninsured_val,teen_birth_val,primary_care_val
0,2009,NaN,26.75,8.92,7845.04,25.88,8.47,27.85,289.72,2.04,...,1.55,17.66,11.89,76.13,11.09,77.93,18.13,14.43,43.22,117.89
1,2010,NaN,25.72,9.29,7816.88,25.35,8.38,28.67,280.81,2.06,...,1.50,19.57,11.39,76.73,12.12,89.86,15.82,15.68,45.61,113.01
2,2011,NaN,27.87,9.23,7603.67,24.64,8.30,28.41,269.24,2.09,...,1.60,18.74,10.50,77.16,12.63,90.42,9.94,15.87,43.19,119.17
3,2012,NaN,29.93,10.21,8014.04,27.18,8.52,28.70,271.80,2.22,...,1.66,19.11,10.43,75.54,12.50,90.22,12.62,16.03,36.42,116.88
4,2013,NaN,31.73,10.20,7635.30,23.72,8.29,28.69,257.98,2.21,...,1.75,18.50,9.92,75.01,13.33,68.63,0.03,15.06,32.90,116.73


In [3]:
race_forecast_df = pd.read_csv('../../../machine_learning/Resources/all_race_2009_2030.csv')
race_forecast_df.head()

,year,mmr_white_hispanic,mmr_white_non_hispanic,mmr_asian_non_hispanic,mmr_black_non_hispanic
0,2009,24.107500,27.388571,NaN,58.173846
1,2010,20.000000,24.042727,16.82,61.764444
2,2011,18.496667,26.372353,NaN,72.118000
3,2012,22.103333,28.588000,NaN,72.866154
4,2013,26.713333,32.740500,13.75,83.244545


In [10]:
for col in race_forecast_df.columns:
    print(f"'{col}': task.{col},")

'year': task.year,
'mmr_white_hispanic': task.mmr_white_hispanic,
'mmr_white_non_hispanic': task.mmr_white_non_hispanic,
'mmr_asian_non_hispanic': task.mmr_asian_non_hispanic,
'mmr_black_non_hispanic': task.mmr_black_non_hispanic,


## Connect to PostgresSQL database

In [4]:
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

In [10]:
# Create table schema
class Race(Base):
    __tablename__ = 'race_data'
    __table_args__ = {'extend_existing': True}
    state = Column(String(255))
    state_code = Column(Integer)
    year = Column(Integer, primary_key=True)
    race = Column(String(255), primary_key=True)
    hispanic_origin = Column(String(255), primary_key=True)
    births_by_race = Column(Integer)
    deaths_by_race = Column(Integer)        
    mmr_by_race = Column(Float)         
    population_by_race = Column(Integer)     
    id = Column(String(255))                   
    state_abbv = Column(String(255), primary_key=True)          
    latitude = Column(Float)            
    longitude = Column(Float)   

In [12]:
class NonRace(Base):
    __tablename__ = 'non_race_data'
    __table_args__ = {'extend_existing': True}
    year = Column(Integer, primary_key=True)  
    state = Column(String(255)) 
    id = Column(String(255))
    state_code = Column(String(255), primary_key=True)
    latitude = Column(Float)
    longitude = Column(Float)
    deaths = Column(Float)
    births = Column(Float)
    maternal_mortality_ratio = Column(Float)
    population = Column(Float)
    employer = Column(Float)
    non_group = Column(Float)
    medicaid = Column(Float)
    medicare = Column(Float)
    military = Column(Float)
    uninsured = Column(Float)
    air_pollution_val = Column(Float)
    cancer_death_val = Column(Float)
    cardio_death_val = Column(Float)
    child_pov_val = Column(Float)   
    choles_check_val = Column(Float) 
    dent_vis_val = Column(Float)    
    dentists_val = Column(Float)    
    diabetes_val = Column(Float)    
    drug_deaths_val = Column(Float)  
    health_stat_fem_val = Column(Float)       
    immun_child_val = Column(Float)  
    income_ineq_val = Column(Float)  
    infant_mort_val = Column(Float)  
    infect_dis_val = Column(Float)   
    obesity_val = Column(Float)      
    phys_inac_val = Column(Float)    
    prem_death_val = Column(Float)   
    smoking_val = Column(Float)      
    uninsured_val = Column(Float)    
    all_determs_val = Column(Float)  
    all_outcomes_val = Column(Float) 
    chlamydia_val = Column(Float)    
    prem_death_ri_val = Column(Float)
    teen_birth_val = Column(Float)   
    primary_care_val = Column(Float) 
    low_birthweight_val = Column(Float)       

In [5]:
class Playground(Base):
    __tablename__ = 'user_input'
    __table_args__ = {'extend_existing': True}
    year = Column(String(255), primary_key=True)
    mmr = Column(Float)

In [35]:
class Forecast(Base):
    __tablename__ = 'ten_year_forecast'
    __table_args__ = {'extend_existing': True}
    year = Column(String(255), primary_key=True)
    mmr_prediction = Column(Float)
    maternal_mortality_ratio = Column(Float)
    diabetes_val = Column(Float)
    prem_death_val = Column(Float)
    phys_inac_val = Column(Float)
    low_birthweight_val = Column(Float)
    obesity_val = Column(Float)
    cardio_death_val = Column(Float)
    medicare = Column(Float)
    cancer_death_val = Column(Float)
    chlamydia_val = Column(Float)
    child_pov_val = Column(Float)
    smoking_val = Column(Float)
    infant_mort_val = Column(Float)
    income_ineq_val = Column(Float)
    dentists_val = Column(Float)
    prem_death_ri_val = Column(Float)
    dent_vis_val = Column(Float)
    all_outcomes_val = Column(Float)
    all_determs_val = Column(Float)
    health_stat_fem_val = Column(Float)
    population = Column(Float)
    employer = Column(Float)
    non_group = Column(Float)
    medicaid = Column(Float)
    military = Column(Float)
    uninsured = Column(Float)
    air_pollution_val = Column(Float)
    choles_check_val = Column(Float)
    drug_deaths_val = Column(Float)
    immun_child_val = Column(Float)
    infect_dis_val = Column(Float)
    uninsured_val = Column(Float)
    teen_birth_val = Column(Float)
    primary_care_val = Column(Float)

In [5]:
class forecastRace(Base):
    __tablename__ = 'ten_year_race_forecast'
    __table_args__ = {'extend_existing': True}
    year = Column(String(255), primary_key=True)
    mmr_white_hispanic = Column(Float)
    mmr_white_non_hispanic = Column(Float)
    mmr_asian_non_hispanic = Column(Float)
    mmr_black_non_hispanic = Column(Float)

In [6]:
# Create Engine
connect_string = f'postgresql://{username}:{password}@{host}/{database}'
engine = create_engine(connect_string)

In [14]:
# Specify table
race_table = [Base.metadata.tables['race_data']]

# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine, tables=race_table)

In [15]:
# Specify table
non_race_table = [Base.metadata.tables['non_race_data']]

# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine, tables=non_race_table)

In [7]:
# Specify table
user_input_table = [Base.metadata.tables['user_input']]

# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine, tables=user_input_table)

In [36]:
# Specify table
forecast_table = [Base.metadata.tables['ten_year_forecast']]

# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine, tables=forecast_table)

In [7]:
# Specify table
race_forecast_table = [Base.metadata.tables['ten_year_race_forecast']]

# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine, tables=race_forecast_table)

In [16]:
# Store data to SQL database
race_df.to_sql(name='race_data', con=engine, if_exists='replace', index=False)

In [17]:
# Store data to SQL database
non_race_df.to_sql(name='non_race_data', con=engine, if_exists='replace', index=False)

In [8]:
avg_data.to_sql(name='user_input', con=engine, if_exists='replace', index=False)

In [37]:
forecast_data.to_sql(name='ten_year_forecast', con=engine, if_exists='replace', index=False)

In [8]:
race_forecast_df.to_sql(name='ten_year_race_forecast', con=engine, if_exists='replace', index=False)

In [18]:
# Check of the table exists in database
pd.read_sql_query('select * from race_data', con=engine).head()

,state,state_code,year,race,hispanic_origin,births_by_race,deaths_by_race,mmr_by_race,population_by_race,id,state_abbv,latitude,longitude
0,Alabama,1,2016,Black or African American,Not Hispanic or Latino,17989,14,77.83,0,US.AL,AL,32.318231,-86.902298
1,Alabama,1,2016,White,Not Hispanic or Latino,35319,18,50.96,0,US.AL,AL,32.318231,-86.902298
2,Alabama,1,2017,Black or African American,Not Hispanic or Latino,18354,25,136.21,704201,US.AL,AL,32.318231,-86.902298
3,Alabama,1,2017,White,Not Hispanic or Latino,34784,15,43.12,1655605,US.AL,AL,32.318231,-86.902298
4,Alabama,1,2018,Black or African American,Not Hispanic or Latino,17939,14,78.04,706754,US.AL,AL,32.318231,-86.902298


In [19]:
# Check of the table exists in database
pd.read_sql_query('select * from non_race_data', con=engine).head()

,year,state,id,state_code,latitude,longitude,deaths,births,maternal_mortality_ratio,population,...,prem_death_val,smoking_val,uninsured_val,all_determs_val,all_outcomes_val,chlamydia_val,prem_death_ri_val,teen_birth_val,primary_care_val,low_birthweight_val
0,2015,Alabama,US.AL,AL,32.318231,-86.902298,12.0,59657.0,20.11,2505795.0,...,10095.0,21.1,12.9,-0.371,-0.325,611.0,1.2,34.3,103.5,10.0
1,2016,Alabama,US.AL,AL,32.318231,-86.902298,35.0,59151.0,59.17,2507714.0,...,10097.0,21.4,11.1,-0.427,-0.366,600.2,1.1,32.0,116.4,10.1
2,2017,Alabama,US.AL,AL,32.318231,-86.902298,41.0,58941.0,69.56,2514911.0,...,10321.0,21.5,9.6,-0.427,-0.335,543.6,1.2,30.1,119.3,10.4
3,2018,Alabama,US.AL,AL,32.318231,-86.902298,31.0,57761.0,53.67,2523756.0,...,10720.0,20.9,9.3,-0.483,-0.356,553.6,1.1,28.4,122.8,10.3
4,2019,Alabama,US.AL,AL,32.318231,-86.902298,35.0,58615.0,59.71,2533668.0,...,10435.0,19.2,9.7,-0.437,-0.383,614.1,1.2,27.0,122.8,10.3


In [9]:
# Check of the table exists in database
pd.read_sql_query('select * from user_input', con=engine)

,year,maternal_mortality_ratio
0,2009,26.75
1,2010,25.72
2,2011,27.87
3,2012,29.93
4,2013,31.73
5,2014,32.34
6,2015,31.94
7,2016,35.44
8,2017,34.21
9,2018,31.53


In [38]:
# Check of the table exists in database
pd.read_sql_query('select * from ten_year_forecast', con=engine).head()

,year,mmr_prediction,maternal_mortality_ratio,diabetes_val,prem_death_val,phys_inac_val,low_birthweight_val,obesity_val,cardio_death_val,medicare,...,military,uninsured,air_pollution_val,choles_check_val,drug_deaths_val,immun_child_val,infect_dis_val,uninsured_val,teen_birth_val,primary_care_val
0,2009,NaN,26.75,8.92,7845.04,25.88,8.47,27.85,289.72,2.04,...,1.55,17.66,11.89,76.13,11.09,77.93,18.13,14.43,43.22,117.89
1,2010,NaN,25.72,9.29,7816.88,25.35,8.38,28.67,280.81,2.06,...,1.50,19.57,11.39,76.73,12.12,89.86,15.82,15.68,45.61,113.01
2,2011,NaN,27.87,9.23,7603.67,24.64,8.30,28.41,269.24,2.09,...,1.60,18.74,10.50,77.16,12.63,90.42,9.94,15.87,43.19,119.17
3,2012,NaN,29.93,10.21,8014.04,27.18,8.52,28.70,271.80,2.22,...,1.66,19.11,10.43,75.54,12.50,90.22,12.62,16.03,36.42,116.88
4,2013,NaN,31.73,10.20,7635.30,23.72,8.29,28.69,257.98,2.21,...,1.75,18.50,9.92,75.01,13.33,68.63,0.03,15.06,32.90,116.73


In [9]:
# Check of the table exists in database
pd.read_sql_query('select * from ten_year_race_forecast', con=engine).head()

,year,mmr_white_hispanic,mmr_white_non_hispanic,mmr_asian_non_hispanic,mmr_black_non_hispanic
0,2009,24.107500,27.388571,NaN,58.173846
1,2010,20.000000,24.042727,16.82,61.764444
2,2011,18.496667,26.372353,NaN,72.118000
3,2012,22.103333,28.588000,NaN,72.866154
4,2013,26.713333,32.740500,13.75,83.244545
